# Краткое описание

Рассмотрим интересный и достаточно интуитивный алгоритм поиска кластеров в графе. Идея основана на плохой сжимаемости жидкости. Для продолжения советуем ознакомиться со статьей, лежащей в этой же папке.

Данный алгоритм отлично подходит для смешанных сообществ. К сожалению на данный момент он реализован только для неорентированных графов. Сам по себе алгоритм явлется вероятностным, поэтому при нескольких запусках разбиения могут отличаться.

#### Плюсы алгоритма

- точное количество задаваемых сообществ
- вероятностный характер (вариативность решений) 
- очень низкая вычислительная сложность - O(E)
- простая интерпретация
- очень хорошо работает для смешанных сообществ

#### Минусы алгоритма

- не учитывает веса
- не реализован для ориентированных графов
- плохо работает для явно-выраженных сообществ с малым количеством связей между сообществами

#### Краткие выводы

Алгоритм просто превосходный. Такая низкая вычислительная сложность позволяет использовать его на графах любого размера. Рекомендуется к использованию при условии неорентированного, смешанного графа. Плохуй работу на явно-выраженных сообществах можно объяснить с помощью физической интерпетации алгоритма. Жидкость быстро занимает кластер и впоследствии выдавить ее через малое количество ребер практически невозможно.

Кратко рассмотрим реализацию алгоритма в библиотеке networkx.

networkx.algorithms.community.asyn_fluid.asyn_fluidc

*asyn_fluidc(G: Graph, k: integer, max_iter: integer =100, seed: integer, random_state, None =None) -> iterable*

- k - количетсво сообществ
- max_iter - максимальное количество итераций
- seed - индикатор для генерации случайного начального состояния
- iterable - сообщества, заданные как наборы узлов

https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.asyn_fluid.asyn_fluidc.html#networkx.algorithms.community.asyn_fluid.asyn_fluidc

In [4]:
from networkx.algorithms import community as cm
import networkx as nx
from matplotlib import pyplot as plt
import igraph as ig
import random as r
from networkx.generators.community import ring_of_cliques

In [5]:
def painter(T, result):
    color_list = get_color(len(result))
    color_map = list()
    buff = T.nodes()
    for node in buff:
        for index in range(len(result)):  
            if node in result[index]:  
                color_map.append(color_list[index])
    plot_graph(T, node_color=color_map)

def get_color(count):
    return [(r.random(), r.random(), r.random(),) for _ in range(count)]

def plot_graph(G, draw_type=None, weight_name=None, node_color='#1f78b4'):
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, width=weights);
    else:
        if draw_type == "circular":
            nx.draw_circular(G, node_color=node_color)
        elif draw_type == "random":
            nx.draw_random(G, node_color=node_color)
        elif draw_type == "spectral":
            nx.draw_spectral(G, node_color=node_color)
        elif draw_type == "spring":
            nx.draw_spring(G, node_color=node_color)
        else:
            nx.draw(G, with_labels=True, node_color=node_color)
    plt.show()

Покажем, что данный алгоритм хорошо работает именно для смешанных сообществ, добавляя на каждой итерации по несколько ребер.

In [ ]:
G = ig.read("ig_test_graph_2.gml")
B = nx.Graph(G.get_edgelist())

count = 3
for _ in range(100):
    F = cm.asyn_fluidc(B, 6, 500)
    print(f'Nodes: {len(B.nodes())}')
    print(f'Edges: {len(B.edges())}')
    node_1 = r.sample(B.nodes(), count)
    node_2 = r.sample(B.nodes(), count)
    for counter in range(count):
        B.add_edge(node_1[counter], node_2[counter])
    x = [y for y in F]
    painter(B, x)

Ради интереса убедимся, что алгоритм по сути своей - поведение жидкости или газа в замкнутой области. Прекрасно видно что на кольце происходит физическое явление - диффузия.

In [ ]:
Z = nx.complete_graph(100)
for counter in range(1, 21):
    F = cm.asyn_fluidc(Z, counter)
    painter(Z, [y for y in F])